In [1]:
import pandas as pd
import fasttext
from sklearn.metrics import f1_score

In [2]:
# 转换为FastText需要的格式
train_df = pd.read_csv('./data/train_set.csv', sep='\t', nrows=15000)
train_df['label_ft'] = '__label__' + train_df['label'].astype(str)
train_df[['text','label_ft']].to_csv('./data/train_fasttext.csv', index=None, header=None, sep='\t')

In [30]:
model = fasttext.train_supervised('./data/train_fasttext.csv',  lr=1.0, wordNgrams=3,
                                  verbose=2, minCount=1, epoch=50, loss="hs")

In [31]:
val_pred = [model.predict(x)[0][0].split('__')[-1] for x in train_df['text']]
print(f1_score(train_df['label'].values.astype(str), val_pred, average='macro'))

0.9999321893792349


In [ ]:
test_df = pd.read_csv('./data/test_a.csv', sep='\t')
test_pred = [model.predict(x)[0][0].split('__')[-1] for x in test_df['text']]

In [ ]:
test_pred = pd.Series(test_pred, name='label', dtype='int8')
test_pred.to_csv('./data/submit_ft.csv', index=False)

In [34]:
model.save_model('./data/fasttext.h5')

fasttext在本地的训练速度很快, 但是提交测试结果时,发现f_score只有0.8495
推测原因是数据量不够大,导致泛化性能不佳, 暂时还没有好的优化思路.